In [1]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook
from bokeh.resources import INLINE
from palettable.colorbrewer import sequential

output_notebook(INLINE)

Loading BokehJS ...

In [2]:
import numpy
import maxr
import h5py

Generate vector field - we need to generate a quiver explicitly using bokeh primitives here

In [4]:
def quiver(x, y, u, v, scale=40, decimate=1, fig=None, ncolors=9, cmap='YlOrRd'):
    """ Generate a quiver plot in bokeh
    """
    # Check that we have gridded x and y
    if len(x.shape) == 1:
        x, y = numpy.meshgrid(y, x)
    
    # Calculate U and V
    if decimate > 1:
        _x, _y = x[::decimate, ::decimate], y[::decimate, ::decimate]
        _u, _v = u[::decimate, ::decimate], v[::decimate, ::decimate]
    else:
        _x, _y = x, y
        _u, _v = u, v
    magnitude = numpy.sqrt(_u * _u + _v * _v).flatten() \
                / float(scale)
    direction = numpy.arctan2(_u, _v).flatten()

    # Construct segments from vector data
    x0s = _x.flatten()
    y0s = _y.flatten()
    x1s = x0s + magnitude * numpy.cos(direction)
    y1s = y0s + magnitude * numpy.sin(direction)
    
    # Construct color map for vectors
    ncolors = int(numpy.clip(ncolors, 3, 9))
    palette = getattr(sequential, '{0}_{1}'.format(cmap, ncolors)).hex_colors
    magmin, magmax = min(magnitude), max(magnitude)
    cindex = lambda m: \
        int((ncolors - 1) * numpy.clip((m - magmin) / (magmax - magmin), 0, 1))
    colors = [palette[cindex(m)] for m in magnitude]

    # Actually generate plots
    plot = fig or figure()
    plot.segment(x0s, y0s, x1s, y1s, color=colors,
                 line_width=2, alpha=0.8)
    return plot

In [5]:
# Define grids
Y, X = numpy.meshgrid(
    numpy.linspace(-3, 3, 100),
    numpy.linspace(-3, 3, 100))

U = -1 - X**2 + Y
V = 1 + X - Y**2

show(quiver(X, Y, U, V, decimate=3, cmap='RdPu'))

We can also construct streamlines for our flow

In [6]:
from maxr.flow import blink, vortex, tick, tock
flow = blink(gamma=1, period=0.5)

In [7]:
ts = numpy.linspace(0, 4, 200)
us, vs = flow(0.2, 0.4, ts)

In [8]:
p = figure(width=500, height=250)
p.line(ts, us, color='red', alpha=0.8)
p.line(ts, vs, alpha=0.8)
show(p)

Make a 3D problem

In [9]:
fname = 'blink.hdf5'
maxr.flow.from_function(flow, fname,
                        xgrid=(-3, 3, 100), ygrid=(-3, 3, 100),
                        tgrid=(0, 4, 100))
flow = maxr.flow.Flow(fname)

In [10]:
from scipy.integrate import ode

def stream_velocity(t, x):
    return [u[0] for u in flow(x[0], x[1], t)]

stream_velocity(0.3, [0.25, 0.25])

solver = ode(stream_velocity)
solver.set_initial_value([0.25, 0.25])

time, path = [], []
t, dt = 0, 0.1
while solver.successful() and t < 5:
    time.append(solver.t + step)
    path.append(solver.integrate(solver.t + step))

TypeError: __call__() takes 2 positional arguments but 4 were given

In [11]:
us.shape

(200,)

In [16]:
index = 8
x, y, u, v = [numpy.asarray(flow.data[k]) for k in ('x', 'y', 'u', 'v')]
show(quiver(x, y, u[..., index], v[..., index], scale=10, decimate=1))